In [4]:
import pandas as pd
from datetime import datetime
import pytz
from scapy.all import Ether, CookedLinux, Raw
import re
import logging
import pyarrow
import random
import os

In [5]:
edt = pytz.timezone('US/Eastern')
def write_log(message):
  current_time = str(datetime.now(edt).strftime('%Y-%m-%d %H:%M:%S'))
  f = open("LOG_CICIDS.txt", "a")
  f.write(current_time + ' : ' + message + '\n')
  f.close()

# Create CSV Files

In [2]:
df = pd.read_csv('CICIDS.csv')

/tmp/ipykernel_1178853/2119090706.py:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CICIDS.csv')


In [3]:
df

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,t_delta,stime_flow,duration,attack_cat,offset,srcip_flow,sport_flow,dstip_flow,dsport_flow,label
0,1499082959,8.254.250.126,80,192.168.10.5,49188,tcp,55,40,Ethernet,b8ac6f360a8b00c1b114eb310800450000281b34000037...,...,0.0,1.499083e+09,4.0,BENIGN,1.0,NaN,NaN,NaN,NaN,0
1,1499082959,8.254.250.126,80,192.168.10.5,49188,tcp,55,40,Ethernet,b8ac6f360a8b00c1b114eb310800450000281b34000037...,...,0.0,1.499083e+09,1.0,BENIGN,1.0,NaN,NaN,NaN,NaN,0
2,1499082982,8.253.185.121,80,192.168.10.14,49486,tcp,55,40,Ethernet,b8ac6f3607ee00c1b114eb310800450000284c22000037...,...,23.0,1.499083e+09,3.0,BENIGN,1.0,NaN,NaN,NaN,NaN,0
3,1499082982,8.253.185.121,80,192.168.10.14,49486,tcp,55,40,Ethernet,b8ac6f3607ee00c1b114eb310800450000284c22000037...,...,23.0,1.499083e+09,1.0,BENIGN,1.0,NaN,NaN,NaN,NaN,0
4,1499082982,8.253.185.121,80,192.168.10.14,49486,tcp,55,40,Ethernet,b8ac6f3607ee00c1b114eb310800450000284c22000037...,...,0.0,1.499083e+09,3.0,BENIGN,1.0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91287885,1499457761,192.168.10.14,59111,23.10.108.151,443,tcp,128,40,Ethernet,00c1b114eb31b8ac6f3607ee0800450000284980400080...,...,0.0,1.499458e+09,113.0,BENIGN,60.0,NaN,NaN,NaN,NaN,0
91287886,1499457761,192.168.10.51,59581,162.213.33.50,443,tcp,64,60,Ethernet,00c1b114eb31b8ac6f360ba808004500003cd6ba400040...,...,0.0,1.499458e+09,191310.0,BENIGN,60.0,NaN,NaN,NaN,NaN,0
91287887,1499457761,192.168.10.51,59581,162.213.33.50,443,tcp,64,52,Ethernet,00c1b114eb31b8ac6f360ba8080045000034d6bb400040...,...,0.0,1.499458e+09,191310.0,BENIGN,60.0,NaN,NaN,NaN,NaN,0
91287888,1499457761,192.168.10.8,10399,184.84.243.218,80,tcp,128,41,Ethernet,00c1b114eb31b8ac6f3608f50800450000297740400080...,...,1.0,1.499458e+09,20082634.0,BENIGN,60.0,NaN,NaN,NaN,NaN,0


In [5]:
chunk = 1
chunksize = 10_000_000

for main_df in pd.read_csv('CICIDS.csv', chunksize=chunksize):
    
    write_log(f'<<<<<<<<----- Started Processing Chunk {chunk} ----->>>>>>>>')
    
    if os.path.isfile(f'./CICIDS/output{chunk}.csv'):
        write_log(f'------------ Skipping DataFrame {chunk} as CSV File already exists ------------')
        chunk += 1
        continue

    packet_info = []
    packet_details = []
    log_records = 100000

    count = 0
    for i in range(len(main_df)):
        packet_type = main_df.iloc[i,8]
        packet_bytes = bytes.fromhex(main_df.iloc[i,9])

        if packet_type == 'cooked linux':
            packet = CookedLinux(packet_bytes)
        elif packet_type == 'Ethernet':
            packet = Ether(packet_bytes)
        else:
            print('Error -> First Layer is not valid')
        details = packet.show(dump=True)
        packet_info.append(details)
        count += 1
        if count%log_records == 0:
          write_log(f'Packets Parsed: {count}')

    write_log(f'------------ All Packets Parsed Successfully for Chunk {chunk} ------------')

    count = 0
    for packet in packet_info:
        fields_values = {}
        current_layer = ""
        for line in packet.split("\n"):
            if line.startswith("###[") and "]" in line:
                current_layer = line.split("]")[0].split("[")[1].strip()
                fields_values[current_layer] = {}
            elif current_layer != "":
                matches = re.findall(r"\s+([a-z_]+)\s+=\s+(.+)", line)
                for match in matches:
                    field_name = match[0]
                    field_value = match[1]
                    fields_values[current_layer][field_name] = field_value
        packet_details.append(fields_values)
        count += 1
        if count%log_records == 0:
          write_log(f'Packet Fields Parsed: {count}')

    write_log(f'------------ All Packet Fields Parsed Successfully for Chunk {chunk} ------------')

    count = 0
    df_list = []
    for fields_values in packet_details:
        row = {}
        for layer, fields in fields_values.items():
            for field in fields:
                column_name = f"{layer} {field}"
                row[column_name] = fields_values[layer][field]
        df_list.append(row)
        count += 1
        if count%log_records == 0:
          write_log(f'Packets appended to DataFrame: {count}')

    df = pd.DataFrame(df_list)

    write_log(f'------------ All Packets appended to the DataFrame {chunk} ------------')

    df = pd.concat([main_df, df], axis=1)
    df.to_csv(f'./CICIDS/output{chunk}.csv', index=False)

    write_log(f'------------ DataFrame {chunk} saved to CSV File ------------')
    
    chunk += 1

/opt/jupyter-hub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Process CSV Files

In [16]:
df_copy = df.copy()

In [17]:
df_copy

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,ARP ptype,ARP hwlen,ARP plen,ARP op,ARP hwsrc,ARP psrc,ARP hwdst,ARP pdst,Kerberos options,Kerberos address
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91287885,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee0800450000284980400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91287886,1.499458e+09,192.168.10.51,59581.0,162.213.33.50,443.0,tcp,64.0,60.0,Ethernet,00c1b114eb31b8ac6f360ba808004500003cd6ba400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91287887,1.499458e+09,192.168.10.51,59581.0,162.213.33.50,443.0,tcp,64.0,52.0,Ethernet,00c1b114eb31b8ac6f360ba8080045000034d6bb400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91287888,1.499458e+09,192.168.10.8,10399.0,184.84.243.218,80.0,tcp,128.0,41.0,Ethernet,00c1b114eb31b8ac6f3608f50800450000297740400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for i in range(10,11):
    if not os.path.isfile(f'./CICIDS/output{i}.csv'):
        continue
    write_log(f'<<<<<<<<----- Started Processing CSV File {i} ----->>>>>>>>')
    df = pd.read_csv(f'./CICIDS/output{i}.csv')
    if df.shape[0]>10_000_000 or i==10:
        write_log(f'------------ Misaligned CSV File {i} Processing ------------')
        rows = df.shape[0]//2
        df1 = df.iloc[rows:,:21]
        df2 = df.iloc[:rows,21:]
        df = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    df.drop_duplicates(inplace=True)
    write_log(f'------------ CSV File {i} Processed. Final shape is {df.shape} ------------')
    df.to_csv(f'./CICIDS/output{i}.csv', index=False)
    write_log(f'------------ CSV File {i} Overwritten and Saved ------------')

/opt/jupyter-hub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,3,5,8,9,10,14,16,18,21,22,23,26,29,32,33,34,35,36,37,42,44,46,47,48,49,51,54,62,67,68,69,70,71,72,74,75,76,77,78,81,82,88,89,91,96,97,98,99,100,101,102,103,104,105,106,107,111,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,157,161,164,165,166,169,170,171,172,173,174,175) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
df = pd.read_csv(f'./CICIDS/output9.csv')

/opt/jupyter-hub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16,18,36,47,48,49,51,52,54,59,60,61,62,63,67,75,80,81,82,83,84,85,86,87,90,91,93,94,95,96,97,98,99,100,101,102,103,104,105,106,108,109,110,111,112,113,114,115,116,122,123,124,125,126,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,154,157,158,159,161,162,163,167,174,175,176,179,180,181,182,183,186,190,195,196,197,198,199,200,203,204,206,207,208,209,210,211,212,213,214,215,216,217,219,220,221,222,225,226,227,228,230,232,234,235,237) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
df

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,ARP ptype,ARP hwlen,ARP plen,ARP op,ARP hwsrc,ARP psrc,ARP hwdst,ARP pdst,Kerberos options,Kerberos address
0,1.499455e+09,192.168.10.50,80.0,172.16.0.1,60079.0,tcp,64.0,2875.0,Ethernet,00c1b114eb310019b90a69f1080045000b3bdf12400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499455e+09,192.168.10.3,53.0,192.168.10.15,51981.0,udp,128.0,117.0,Ethernet,001e4fd4ca281866da9be37d0800450000752396000080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499455e+09,192.168.10.50,80.0,172.16.0.1,60079.0,tcp,64.0,8800.0,Ethernet,00c1b114eb310019b90a69f1080045002260df0c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499455e+09,192.168.10.50,80.0,172.16.0.1,35678.0,tcp,64.0,8715.0,Ethernet,00c1b114eb310019b90a69f108004500220b14cf400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499455e+09,192.168.10.50,80.0,172.16.0.1,35678.0,tcp,64.0,2960.0,Ethernet,00c1b114eb310019b90a69f1080045000b9014cd400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287885,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee0800450000284980400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287886,1.499458e+09,192.168.10.51,59581.0,162.213.33.50,443.0,tcp,64.0,60.0,Ethernet,00c1b114eb31b8ac6f360ba808004500003cd6ba400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287887,1.499458e+09,192.168.10.51,59581.0,162.213.33.50,443.0,tcp,64.0,52.0,Ethernet,00c1b114eb31b8ac6f360ba8080045000034d6bb400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287888,1.499458e+09,192.168.10.8,10399.0,184.84.243.218,80.0,tcp,128.0,41.0,Ethernet,00c1b114eb31b8ac6f3608f50800450000297740400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.iloc[0].dropna()

stime                                                 1499454694.0
srcip                                                192.168.10.50
sport                                                         80.0
dstip                                                   172.16.0.1
dsport                                                     60079.0
protocol_m                                                     tcp
sttl                                                          64.0
total_len                                                   2875.0
first_layer                                               Ethernet
packet           00c1b114eb310019b90a69f1080045000b3bdf12400040...
payload          6d616e2e6367693f71756572793d613264697373697465...
t_delta                                                        0.0
stime_flow                                            1499454660.0
duration                                                  581642.0
attack_cat                                                    

In [21]:
df2

,TCP flags,TCP window,TCP chksum,TCP urgptr,TCP options,Raw load,UDP sport,UDP dport,UDP len,UDP chksum,...,ARP ptype,ARP hwlen,ARP plen,ARP op,ARP hwsrc,ARP psrc,ARP hwdst,ARP pdst,Kerberos options,Kerberos address
0,PA,229.0,0x8219,0.0,[],"'man.cgi?query=a2dissite"">a2dissite</a>,\n ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,domain,51981,97.0,0x2c5f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,229.0,0x993e,0.0,[],"'HTTP/1.1 200 OK\r\nDate: Fri, 07 Jul 2017 19:...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PA,229.0,0x98e9,0.0,[],'ng_element {\n position: relative;\n fl...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,229.0,0x826e,0.0,[],"'HTTP/1.1 200 OK\r\nDate: Fri, 07 Jul 2017 19:...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287885,FA,256.0,0x7d89,0.0,'',NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287886,S,29200.0,0xd09d,0.0,"[('MSS', 1460), ('SAckOK', b''), ('Timestamp',...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287887,A,229.0,0x3b49,0.0,"[('NOP', None), ('NOP', None), ('Timestamp', (...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287888,A,16329.0,0x4de2,0.0,[],'\x00',NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df1

,Ethernet dst,Ethernet src,Ethernet type,IP version,IP ihl,IP tos,IP len,IP id,IP flags,IP frag,...,IP proto,IP chksum,IP src,IP dst,TCP sport,TCP dport,TCP seq,TCP ack,TCP dataofs,TCP reserved
1287890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2575776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2575777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2575778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df2.shape[0]//2

1287890

In [23]:
df

,Ethernet dst,Ethernet src,Ethernet type,IP version,IP ihl,IP tos,IP len,IP id,IP flags,IP frag,...,ARP ptype,ARP hwlen,ARP plen,ARP op,ARP hwsrc,ARP psrc,ARP hwdst,ARP pdst,Kerberos options,Kerberos address
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Sample Records

In [3]:
for i in range(0,15):
    if not os.path.isfile(f'./CICIDS/output{i}.csv'):
        continue
    write_log(f'<<<<<<<<----- Started Processing CSV File {i} ----->>>>>>>>')
    df = pd.read_csv(f'./CICIDS/output{i}.csv')
    df = df[(df['protocol_m'] == 'tcp') | (df['protocol_m'] == 'udp')]
    df = df.dropna(subset=['payload'])
    df = df.dropna(axis=1, how='all')
    write_log(f'------------ CSV File {i} Processed. Final shape is {df.shape} ------------')
    df.to_csv(f'./CICIDS/CICIDS-1/output{i}.csv', index=False)
    write_log(f'------------ CSV File {i} Saved ------------')

/opt/jupyter-hub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16,18,36,47,48,50,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,72,80,85,86,87,88,89,90,92,96,97,98,99,100,101,102,103,104,105,106,107,112,116,120,121,122,123,124,125,126,127,128,132,133,135,138,142,147,148,149,150,151,152,153,154,155,157,158,159,160,161,162,171,173,178,179,180,181,182,183,184,185,187,188,190,191,192,193,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,226,231,232,233,234,235,237,238,239,240,243) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/jupyter-hub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16,18,37,47,49,50,52,55,63,68,69,70,71,72,73,75,76,77,78,79,82,83,84,85,86,87,88,90,95,96,97,98,99,100,101,102,108,112,113,114,115,116,117,118,119,120,121,122,123,124,125,127,128,129,130,131,

# Concatenate Files

In [ ]:
k = 1
df = pd.DataFrame()
for i in range(0,12):
    if not os.path.isfile(f'./CICIDS/CICIDS-1/output{i}.csv'):
        continue
    write_log(f'<<<<<<<<----- Started Reading CSV File {i} ----->>>>>>>>')
    temp_df = pd.read_csv(f'./CICIDS/CICIDS-1/output{i}.csv')
    df = pd.concat([df, temp_df], ignore_index=True)
    write_log(f'------------ CSV File {i} added to DataFrame ------------')
    if (i%3 == 0 and i!=9) or i==10:
        df.to_csv(f'./CICIDS/CICIDS-2/output{k}.csv', index=False)
        print(df['attack_cat'].value_counts())
        df = pd.DataFrame()
        k = k+1

# Sampling DataFrame

In [21]:
df = pd.read_csv('./CICIDS/CICIDS-2/output3.csv')

/tmp/ipykernel_3856262/1395569476.py:1: DtypeWarning: Columns (16,18,36,37,48,49,51,52,54,59,60,61,62,63,64,65,68,76,81,82,83,84,85,86,87,88,89,92,94,95,96,102,103,104,105,106,107,109,110,111,112,113,114,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,135,136,137,138,139,140,149,151,154,155,156,157,158,159,160,161,162,163,164,167,171,176,177,178,179,180,181,182,188,191,192,193,197,198,199,201,203,206,207,208,209,210,211,212,213,214,215) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./CICIDS/CICIDS-2/output3.csv')


In [22]:
df

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,PPTP Start Control Connection Request host_name,Session Setup AndX Extended Security Response (SMB) pvno,Session Setup AndX Extended Security Response (SMB) etype,Session Setup AndX Extended Security Response (SMB) kvno,Session Setup AndX Extended Security Response (SMB) cipher,Session Setup AndX Extended Security Request (SMB) pvno,Session Setup AndX Extended Security Request (SMB) realm,Session Setup AndX Extended Security Request (SMB) etype,Session Setup AndX Extended Security Request (SMB) kvno,Session Setup AndX Extended Security Request (SMB) cipher
0,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d01400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b903d0f400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d01400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d02400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b903d0d400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25787313,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee080045000028497f400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787314,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee080045000028497f400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787315,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee0800450000284980400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787316,1.499458e+09,192.168.10.14,59111.0,23.10.108.151,443.0,tcp,128.0,40.0,Ethernet,00c1b114eb31b8ac6f3607ee0800450000284980400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df['attack_cat'].value_counts() # Part 2

BENIGN              11612298
DoS Hulk             2286837
SSH-Patator           138639
Heartbleed             41285
DoS GoldenEye          36151
DoS slowloris          25139
DoS Slowhttptest       11641
Name: attack_cat, dtype: int64

In [7]:
df['attack_cat'].value_counts() # Part 1

BENIGN         15231102
FTP-Patator      110636
SSH-Patator       42951
Name: attack_cat, dtype: int64

In [23]:
df = df[df['protocol_m']=='tcp']
columns_to_remove = df.columns[df.notna().sum() < 1000]
rows_to_remove = df[df[columns_to_remove].notna().any(axis=1)].index
df = df.drop(columns_to_remove, axis=1)
df = df.drop(rows_to_remove)
df = df.dropna(subset='Raw load')

In [24]:
df

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,DCE/RPC v5 rpc_vers,DCE/RPC v5 ptype,DCE/RPC v5 pfc_flags,DCE/RPC v5 endian,DCE/RPC v5 encoding,DCE/RPC v5 float,DCE/RPC v5 frag_len,DCE/RPC v5 auth_len,DCE/RPC v5 call_id,SMB2 Negotiate Protocol Response load
0,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d01400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b903d0f400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d01400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3d02400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b903d0d400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25787305,1.499458e+09,192.168.10.50,22.0,192.168.10.17,37759.0,tcp,64.0,900.0,Ethernet,0023ae9b95670019b90a69f108004500038442a9400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787310,1.499458e+09,23.10.108.151,443.0,192.168.10.14,59111.0,tcp,54.0,71.0,Ethernet,b8ac6f3607ee00c1b114eb31080045000047aaf3400036...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787311,1.499458e+09,23.10.108.151,443.0,192.168.10.14,59111.0,tcp,54.0,71.0,Ethernet,b8ac6f3607ee00c1b114eb31080045000047aaf3400036...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25787312,1.499458e+09,192.168.10.51,59581.0,162.213.33.50,443.0,tcp,64.0,200.0,Ethernet,00c1b114eb31b8ac6f360ba80800450000c8d6bc400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df['attack_cat'].value_counts() # Part 3

BENIGN                        14235590
DDoS                            619353
Infiltration                     56242
Web Attack – Brute Force         28922
Web Attack – XSS                  6767
Bot                               5147
PortScan                           838
Web Attack – Sql Injection          45
Name: attack_cat, dtype: int64

In [27]:
fdf = pd.DataFrame()

In [41]:
sdf = df[df['attack_cat']=='Bot']

In [33]:
sdf = df[df['attack_cat']=='Infiltration'].sample(50000, ignore_index=True)

In [42]:
fdf = pd.concat([fdf, sdf], ignore_index=True)

In [43]:
fdf

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,DCE/RPC v5 rpc_vers,DCE/RPC v5 ptype,DCE/RPC v5 pfc_flags,DCE/RPC v5 endian,DCE/RPC v5 encoding,DCE/RPC v5 float,DCE/RPC v5 frag_len,DCE/RPC v5 auth_len,DCE/RPC v5 call_id,SMB2 Negotiate Protocol Response load
0,1.499453e+09,69.4.95.11,80.0,192.168.10.15,58402.0,tcp,50.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc05fe400032...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b9038ad400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc6449400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b907fa1400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b90366a400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156714,1.499436e+09,192.168.10.8,3051.0,205.174.165.73,8080.0,tcp,128.0,238.0,Ethernet,00c1b114eb31b8ac6f3608f50800450000ee2e9f400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156715,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156716,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156717,1.499437e+09,192.168.10.17,48034.0,52.7.235.158,8080.0,tcp,64.0,465.0,Ethernet,00c1b114eb310023ae9b95670800450001d19a5c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
fdf['attack_cat'].value_counts()

DDoS                          50000
Infiltration                  50000
Web Attack – Brute Force      28922
BENIGN                        15000
Web Attack – XSS               6767
Bot                            5147
PortScan                        838
Web Attack – Sql Injection       45
Name: attack_cat, dtype: int64

In [45]:
fdf.to_csv(f'./CICIDS/CICIDS-3/output3.csv', index=False)

# Combining CSV Files

In [46]:
df1 = pd.read_csv('./CICIDS/CICIDS-3/output1.csv')

/tmp/ipykernel_3856262/2729298540.py:1: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('./CICIDS/CICIDS-3/output1.csv')


In [47]:
df1

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,DCE/RPC v5 frag_len,DCE/RPC v5 auth_len,DCE/RPC v5 call_id,LDAP referral,Kerberos from_,Kerberos till,Kerberos rtime,Kerberos nonce,LDAP scope,LDAP present
0,1.499171e+09,13.107.4.50,80.0,192.168.10.15,50580.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc2d27400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499096e+09,35.186.253.0,443.0,192.168.10.25,55458.0,tcp,45.0,1638.0,Ethernet,002500a8c46000c1b114eb31080045000666586800002d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499085e+09,13.107.4.50,80.0,192.168.10.15,49647.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc45f6400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499173e+09,8.253.104.14,80.0,192.168.10.15,51929.0,tcp,54.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dcaec5000036...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499171e+09,13.107.4.50,80.0,192.168.10.14,50128.0,tcp,119.0,2960.0,Ethernet,b8ac6f3607ee00c1b114eb31080045000b905f18400077...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,1.499189e+09,192.168.10.50,22.0,172.16.0.1,47108.0,tcp,64.0,116.0,Ethernet,00c1b114eb310019b90a69f1080045000074cc4c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84996,1.499189e+09,172.16.0.1,47162.0,192.168.10.50,22.0,tcp,62.0,164.0,Ethernet,0019b90a69f100c1b114eb310800450000a4a9a340003e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84997,1.499188e+09,172.16.0.1,46588.0,192.168.10.50,22.0,tcp,62.0,324.0,Ethernet,0019b90a69f100c1b114eb31080045000144d14d40003e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84998,1.499188e+09,192.168.10.50,22.0,172.16.0.1,46842.0,tcp,64.0,93.0,Ethernet,00c1b114eb310019b90a69f108004500005dc135400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df2 = pd.read_csv('./CICIDS/CICIDS-3/output2.csv')

/tmp/ipykernel_3856262/586693513.py:1: DtypeWarning: Columns (16,18,36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./CICIDS/CICIDS-3/output2.csv')


In [49]:
df2

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,LDAP mechanism,DCE/RPC v5 rpc_vers,DCE/RPC v5 ptype,DCE/RPC v5 pfc_flags,DCE/RPC v5 endian,DCE/RPC v5 encoding,DCE/RPC v5 float,DCE/RPC v5 frag_len,DCE/RPC v5 auth_len,DCE/RPC v5 call_id
0,1.499193e+09,185.49.134.3,443.0,192.168.10.17,33666.0,tcp,48.0,1500.0,Ethernet,0023ae9b956700c1b114eb310800450005dc4985400030...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499256e+09,13.107.4.50,80.0,192.168.10.15,49672.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc15e3400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499190e+09,52.84.145.229,443.0,192.168.10.25,60532.0,tcp,245.0,1500.0,Ethernet,002500a8c46000c1b114eb310800450005dc630f4000f5...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499256e+09,13.107.4.50,80.0,192.168.10.15,49672.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc3432400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499256e+09,13.107.4.50,80.0,192.168.10.15,49672.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc1e87400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214211,1.499280e+09,192.168.10.51,444.0,172.16.0.1,45022.0,tcp,64.0,5844.0,Ethernet,00c1b114eb31b8ac6f360ba80800450016d42b3d400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214212,1.499280e+09,192.168.10.51,444.0,172.16.0.1,45022.0,tcp,64.0,1500.0,Ethernet,00c1b114eb31b8ac6f360ba80800450005dc2b3c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214213,1.499280e+09,192.168.10.51,444.0,172.16.0.1,45022.0,tcp,64.0,1500.0,Ethernet,00c1b114eb31b8ac6f360ba80800450005dc2b3c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214214,1.499280e+09,192.168.10.51,444.0,172.16.0.1,45022.0,tcp,64.0,5844.0,Ethernet,00c1b114eb31b8ac6f360ba80800450016d42b38400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df3 = pd.read_csv('./CICIDS/CICIDS-3/output3.csv')

/tmp/ipykernel_3856262/3806815431.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('./CICIDS/CICIDS-3/output3.csv')


In [51]:
df3

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,DCE/RPC v5 rpc_vers,DCE/RPC v5 ptype,DCE/RPC v5 pfc_flags,DCE/RPC v5 endian,DCE/RPC v5 encoding,DCE/RPC v5 float,DCE/RPC v5 frag_len,DCE/RPC v5 auth_len,DCE/RPC v5 call_id,SMB2 Negotiate Protocol Response load
0,1.499453e+09,69.4.95.11,80.0,192.168.10.15,58402.0,tcp,50.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc05fe400032...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b9038ad400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc6449400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499430e+09,13.107.4.50,80.0,192.168.10.15,49808.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b907fa1400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499343e+09,13.107.4.50,80.0,192.168.10.15,49910.0,tcp,118.0,2960.0,Ethernet,001e4fd4ca2800c1b114eb31080045000b90366a400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156714,1.499436e+09,192.168.10.8,3051.0,205.174.165.73,8080.0,tcp,128.0,238.0,Ethernet,00c1b114eb31b8ac6f3608f50800450000ee2e9f400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156715,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156716,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156717,1.499437e+09,192.168.10.17,48034.0,52.7.235.158,8080.0,tcp,64.0,465.0,Ethernet,00c1b114eb310023ae9b95670800450001d19a5c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [53]:
df

,stime,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,first_layer,packet,...,LDAP referral,Kerberos from_,Kerberos till,Kerberos rtime,Kerberos nonce,LDAP scope,LDAP present,SMB Negotiate Extended Security Response (SMB) load,SMB2 LOGOFF Request reserved,SMB2 Negotiate Protocol Response load
0,1.499171e+09,13.107.4.50,80.0,192.168.10.15,50580.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc2d27400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.499096e+09,35.186.253.0,443.0,192.168.10.25,55458.0,tcp,45.0,1638.0,Ethernet,002500a8c46000c1b114eb31080045000666586800002d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.499085e+09,13.107.4.50,80.0,192.168.10.15,49647.0,tcp,118.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dc45f6400076...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.499173e+09,8.253.104.14,80.0,192.168.10.15,51929.0,tcp,54.0,1500.0,Ethernet,001e4fd4ca2800c1b114eb310800450005dcaec5000036...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.499171e+09,13.107.4.50,80.0,192.168.10.14,50128.0,tcp,119.0,2960.0,Ethernet,b8ac6f3607ee00c1b114eb31080045000b905f18400077...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455930,1.499436e+09,192.168.10.8,3051.0,205.174.165.73,8080.0,tcp,128.0,238.0,Ethernet,00c1b114eb31b8ac6f3608f50800450000ee2e9f400080...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455931,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455932,1.499437e+09,52.7.235.158,8080.0,192.168.10.17,48034.0,tcp,237.0,308.0,Ethernet,0023ae9b956700c1b114eb31080045000134db614000ed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455933,1.499437e+09,192.168.10.17,48034.0,52.7.235.158,8080.0,tcp,64.0,465.0,Ethernet,00c1b114eb310023ae9b95670800450001d19a5c400040...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df['attack_cat'].value_counts()

BENIGN                        50000
FTP-Patator                   50000
SSH-Patator                   50000
DoS Hulk                      50000
DDoS                          50000
Infiltration                  50000
Heartbleed                    41285
DoS GoldenEye                 36151
Web Attack – Brute Force      28922
DoS slowloris                 25139
DoS Slowhttptest              11641
Web Attack – XSS               6767
Bot                            5147
PortScan                        838
Web Attack – Sql Injection       45
Name: attack_cat, dtype: int64

In [55]:
df.to_csv('./CICIDS/CICIDS-4/output.csv', index=False)

# EDA

In [ ]:
k = 1
df = pd.DataFrame()
for i in range(0,12):
    if not os.path.isfile(f'./CICIDS/output{i}.csv'):
        continue
    write_log(f'<<<<<<<<----- Started Reading CSV File {i} ----->>>>>>>>')
    temp_df = pd.read_csv(f'./CICIDS/output{i}.csv')
    df = pd.concat([df, temp_df], ignore_index=True)
    write_log(f'------------ CSV File {i} added to DataFrame ------------')
    if (i%3 == 0 and i!=9) or i==10:
        df.to_csv(f'./CICIDS/CICIDS-EDA/output{k}.csv', index=False)
        print(df['attack_cat'].value_counts())
        df = pd.DataFrame()
        k = k+1

/tmp/ipykernel_3710999/511977792.py:7: DtypeWarning: Columns (16,18,36,47,48,50,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,72,80,85,86,87,88,89,90,92,96,97,98,99,100,101,102,103,104,105,106,107,112,116,120,121,122,123,124,125,126,127,128,132,133,135,138,142,147,148,149,150,151,152,153,154,155,157,158,159,160,161,162,171,173,178,179,180,181,182,183,184,185,187,188,190,191,192,193,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,226,231,232,233,234,235,237,238,239,240,243) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'./CICIDS/output{i}.csv')
/tmp/ipykernel_3710999/511977792.py:7: DtypeWarning: Columns (16,18,37,47,49,50,52,55,63,68,69,70,71,72,73,75,76,77,78,79,82,83,84,85,86,87,88,90,95,96,97,98,99,100,101,102,108,112,113,114,115,116,117,118,119,120,121,122,123,124,125,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,146,150,153,154,155,156,157,160,161,162,164,165,16

# Exporting Files

In [ ]:
def hex_to_dec(hex_str):
    try:
        return [int(hex_str[i:i+2], 16) for i in range(0, len(hex_str), 2)]
    except ValueError:
        return [0,0]

start_value = 16952481
k = 5
flow1 = pd.read_csv('./CICIDS/Export/CICIDS_Flow.csv')
flow1 = flow1[['flow_id', 'source_ip', 'source_port', 'destination_ip', 'destination_port', 'protocol']]
flow2 = flow1.copy()
flow2.rename(columns={'source_ip': 'destination_ip', 'destination_ip': 'source_ip', 'source_port': 'destination_port', 'destination_port': 'source_port'}, inplace=True)
flow = pd.concat([flow1, flow2])
flow.drop_duplicates(subset=flow.columns.difference(['flow_id']), inplace=True)

for i in range(3, 20):
    
    if not os.path.isfile(f'./CICIDS/output{i}.csv'):
        continue
    write_log(f'<<<<<<<<----- Started Processing CSV File {i} ----->>>>>>>>\n')
    main_df = pd.read_csv(f'./CICIDS/output{i}.csv')
    for j in range(2):
        if (i==10 or i==6) and j==0:
            df = main_df
        elif (i==10 or i==6) and j==1:
            continue
        else:
            rows = len(main_df)//2
            if j==0:
                df = main_df.head(rows)
            elif j==1:
                df = main_df.tail(len(main_df)-rows)
        write_log(f'------------ BEGIN PACKET FLOW MERGE ------------')
        df.rename(columns={'packet': 'packet_hex', 'payload': 'payload_hex', 'srcip': 'source_ip', 'dstip': 'destination_ip', 'sport': 'source_port', 'dsport': 'destination_port', 'protocol_m': 'protocol'}, inplace=True)
        df['source_port'] = df['source_port'].astype(int)
        df['destination_port'] = df['destination_port'].astype(int)
        columns_to_match = ['source_ip', 'source_port', 'destination_ip', 'destination_port', 'protocol']
        df = df.merge(flow, on=columns_to_match, how='left')
        flow_id = df.pop('flow_id')
        df.insert(0, 'flow_id', flow_id)
        end_value = start_value + len(df)
        df['packet_id'] = range(start_value, end_value)
        df['payload_length'] = df.payload_hex.apply(lambda x: len(x)//2 if isinstance(x, str) else 0)
        write_log(f'------------ END PACKET FLOW MERGE ------------')

        write_log(f'------------ BEGIN PAYLOAD BYTES PROCESSING ------------')
        df1 = df.dropna(subset='Raw load')
        df1.reset_index(drop=True, inplace=True)
        dec_data = df1['payload_hex'].apply(lambda x: hex_to_dec(str(x)[:3000]))
        max_len = dec_data.apply(len).max()
        df_final = pd.DataFrame(dec_data.tolist(), columns=[f'payload_byte_{i}' for i in range(1,max_len+1)])
        df_final = pd.concat([df1[['packet_id', 'flow_id', 'source_ip', 'source_port', 'destination_ip', 'destination_port', 'protocol', 'payload_length']], df_final], axis=1)
        df_final['attack_label'] = df1['attack_cat']
        df_final.drop_duplicates(subset=df_final.columns.difference(['packet_id', 'flow_id']), inplace=True)
        write_log(f'------------ END PAYLOAD BYTES PROCESSING ------------')

        df_final.to_csv(f'./CICIDS/Export/Payload-Bytes/Payload_Bytes_File_{k}.csv', index=False)
        write_log(f'------------ CSV File {i}:{j}:{k} Saved for Payload Bytes ------------')

        del dec_data
        del df_final
        del df1
        
        write_log(f'------------ BEGIN PACKET BYTES PROCESSING ------------')
        dec_data = df['packet_hex'].apply(lambda x: hex_to_dec(x[:3200]))
        max_len = dec_data.apply(len).max()
        df_final = pd.DataFrame(dec_data.tolist(), columns=[f'packet_byte_{i}' for i in range(1,max_len+1)])
        df_final = pd.concat([df[['packet_id', 'flow_id', 'source_ip', 'source_port', 'destination_ip', 'destination_port', 'protocol', 'payload_length']], df_final], axis=1)
        df_final['attack_label'] = df['attack_cat']
        df_final.drop_duplicates(subset=df_final.columns.difference(['packet_id', 'flow_id']), inplace=True)
        write_log(f'------------ END PACKET BYTES PROCESSING ------------')

        df_final.to_csv(f'./CICIDS/Export/Packet-Bytes/Packet_Bytes_File_{k}.csv', index=False)
        write_log(f'------------ CSV File {i}:{j}:{k} Saved for Packet Bytes ------------')
        
        del dec_data
        del df_final

        write_log(f'------------ BEGIN PACKET FIELDS PROCESSING ------------')
        df.drop(['stime', 'sttl', 'total_len', 'first_layer', 't_delta', 'stime_flow', 'duration', 'offset', 'srcip_flow', 'sport_flow', 'dstip_flow', 'dsport_flow', 'label', 'payload_length'], axis=1, inplace=True)
        attack_label = df.pop('attack_cat')
        df.insert(len(df.columns), 'attack_label', attack_label)
        packet_id = df.pop('packet_id')
        df.insert(0, 'packet_id', packet_id)
        df.drop_duplicates(subset=df.columns.difference(['packet_id', 'flow_id']), inplace=True)
        start_value += len(df)
        counts = '\n'.join([f'{key}:{value}' for key, value in df.attack_label.value_counts().to_dict().items()])
        f = open("CICIDS_INFO.txt", "a")
        f.write(f'TOTAL PACKETS IN CSV FILE {k}: {df.shape}\n')
        f.write(f'ATTACK LABELS IN CSV FILE {k}\n' + counts + '\n\n')
        f.close()
        write_log(f'------------ END PACKET FIELDS PROCESSING ------------')

        df.to_csv(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{k}.csv', index=False)
        write_log(f'------------ CSV File {i}:{j}:{k} Saved for Packet Fields ------------\n')
        k+=1
        
        del df
        
    del main_df

In [ ]:
for i in range(1,19):
    file_name_1 = f"./CICIDS/Export/Payload-Bytes/Payload_Bytes_File_{i}.zip"
    !unzip {file_name_1}
    
    file_name_2 = f"./CICIDS/Export/Packet-Bytes/Packet_Bytes_File_{i}.zip"
    !unzip {file_name_2}
    
    file_name_3 = f"./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.zip"
    !unzip {file_name_3}
    
    write_log(f"FILE {i} UNZIPPED SUCCESSFULLY")

# Uploading Files to Hugging Face

In [1]:
from huggingface_hub import login, HfApi

In [2]:
login()

In [3]:
api = HfApi()

In [4]:
api.upload_folder(
    folder_path="./CICIDS/Export/Packet-Fields",
    repo_id="rdpahalavan/CIC-IDS2017",
    path_in_repo="Packet-Fields/",
    repo_type="dataset",
    allow_patterns="*.parquet",
    delete_patterns="*.parquet",
    multi_commits=True,
    multi_commits_verbose=True
)

/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/huggingface_hub/utils/_experimental.py:57: UserWarning: 'plan_multi_commits' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/huggingface_hub/utils/_experimental.py:57: UserWarning: 'HfApi.create_commits_on_pr' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
Will create 0 deletion commit(s) and 18 addition commit(s), totalling 18 atomic operations.
Multi-commits strategy with ID 7c904ad71385d209b0053f17d71f906afb351392bb35333dd1883c882405794c.
PR already exists: https://huggingface.co/datasets/rdpahalavan/CIC-IDS2017/discussions/6. Will resume process where it stopped.
Found 9 e

Packet_Fields_File_2.parquet:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

  step 918501eb5bdfdfcdd0ad2c6ee9b89eee6bb4a1b11e137f3a0fce0eabb47d979e completed (still 8 to go).


Packet_Fields_File_3.parquet:   0%|          | 0.00/13.6G [00:00<?, ?B/s]

  step 147fac4088ff613c70a50a1f31669161860e9281c10ae7c7f03d62cb47dd32da completed (still 7 to go).


...

Packet_Fields_File_9.parquet:   0%|          | 0.00/19.9G [00:00<?, ?B/s]

  step b3b2d9e68d720c614ad5f0e8ca7a720ee6ec146e007f3335f48dc20e874b9b8b completed (still 1 to go).


Packet_Fields_File_18.parquet:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

  step d0e05e89fcb38a137c5e27506454ba93471d7ac662932ea8de41da5612bffb61 completed (still 0 to go).
All commits have been pushed.
PR is now open for reviews.
PR has been automatically merged (`merge_pr=True` was passed).


'https://huggingface.co/datasets/rdpahalavan/CIC-IDS2017/tree/main/Packet-Fields/'

# CSV to Parquet

In [ ]:
for i in range(1,19):
    write_log(f'<<<<<<<<----- Started Processing CSV File {i} ----->>>>>>>>\n')
    df = pd.read_csv(f'./CICIDS/Export/Payload-Bytes/Payload_Bytes_File_{i}.csv', low_memory=False)
    df = df.convert_dtypes()
    df.to_csv(f'./CICIDS/Export/Payload-Bytes/Payload_Bytes_File_{i}.csv', index=False)
    df.to_parquet(f'./CICIDS/Export/Payload-Bytes/Payload_Bytes_File_{i}.parquet', index=False)
    del df
    write_log(f'------------ Payload Bytes Exported ------------')
    df = pd.read_csv(f'./CICIDS/Export/Packet-Bytes/Packet_Bytes_File_{i}.csv', low_memory=False)
    df = df.convert_dtypes()
    df.to_csv(f'./CICIDS/Export/Packet-Bytes/Packet_Bytes_File_{i}.csv', index=False)
    df.to_parquet(f'./CICIDS/Export/Packet-Bytes/Packet_Bytes_File_{i}.parquet', index=False)
    del df
    write_log(f'------------ Packet Bytes Exported ------------')
    df = pd.read_csv(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.csv', low_memory=False)
    df = df.convert_dtypes()
    df.to_csv(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.csv', index=False)
    df.to_parquet(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.parquet', index=False)
    del df
    write_log(f'------------ Packet Fields Exported ------------')

In [6]:
df = pd.read_csv(f'./CICIDS/Export/Network-Flows/CICIDS_Flow.csv', low_memory=False)
df = df.convert_dtypes()
df.to_csv(f'./CICIDS/Export/Network-Flows/CICIDS_Flow.csv', index=False)
df.to_parquet(f'./CICIDS/Export/Network-Flows/CICIDS_Flow.parquet', index=False)

In [1]:
import pandas as pd
import numpy as np

all_columns = set()

for i in range(1,19):
    df = pd.read_csv(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.csv', nrows=0)
    all_columns = all_columns.union(df.columns)

In [2]:
len(all_columns)

261

In [3]:
for i in range(1,19):
    df = pd.read_parquet(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.parquet')
    missing_cols = all_columns - set(df.columns)
    df_missing_cols = pd.DataFrame({col: np.nan for col in missing_cols}, index=df.index)
    df_missing_cols = df_missing_cols.convert_dtypes()
    df = pd.concat([df, df_missing_cols], axis=1)
    df.to_parquet(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.parquet', index=False)
    del df_missing_cols
    del df
    print(f"Completed: {i}")

Completed: 1
Completed: 2
Completed: 3
Completed: 4
Completed: 5
Completed: 6
Completed: 7
Completed: 8
Completed: 9
Completed: 10
Completed: 11
Completed: 12
Completed: 13
Completed: 14
Completed: 15
Completed: 16
Completed: 17
Completed: 18


In [3]:
import pandas as pd
import pyarrow.parquet as pq
from collections import defaultdict

files = [f"Packet_Fields_File_{i}.parquet" for i in range(1,19)]

column_types = defaultdict(list)

for file in files:
    df = pd.read_parquet("./CICIDS/Export/Packet-Fields/"+file)
    for column, dtype in df.dtypes.items():
        column_types[column].append(dtype)
    print(f"Completed: {file}")

Completed: Packet_Fields_File_1.parquet
Completed: Packet_Fields_File_2.parquet
Completed: Packet_Fields_File_3.parquet
Completed: Packet_Fields_File_4.parquet
Completed: Packet_Fields_File_5.parquet
Completed: Packet_Fields_File_6.parquet
Completed: Packet_Fields_File_7.parquet
Completed: Packet_Fields_File_8.parquet
Completed: Packet_Fields_File_9.parquet
Completed: Packet_Fields_File_10.parquet
Completed: Packet_Fields_File_11.parquet
Completed: Packet_Fields_File_12.parquet
Completed: Packet_Fields_File_13.parquet
Completed: Packet_Fields_File_14.parquet
Completed: Packet_Fields_File_15.parquet
Completed: Packet_Fields_File_16.parquet
Completed: Packet_Fields_File_17.parquet
Completed: Packet_Fields_File_18.parquet


In [4]:
from collections import Counter

majority_column_types = {}

for column, types in column_types.items():
    counter = Counter(types)
    majority_type = counter.most_common(1)[0][0]
    majority_column_types[column] = majority_type

In [21]:
for i in range(1,19):
    df = pd.read_parquet(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.parquet')
    for column, dtype in df.dtypes.items():
        majority_type = majority_column_types[column]
        if dtype != majority_type:
            df[column] = df[column].astype(majority_type)
    df.to_parquet(f'./CICIDS/Export/Packet-Fields/Packet_Fields_File_{i}.parquet', index=False)
    del df
    print(f"Completed: {i}")

Completed: 1
Completed: 2
Completed: 3
Completed: 4
Completed: 5
Completed: 6
Completed: 7
Completed: 8
Completed: 9
Completed: 10
Completed: 11
Completed: 12
Completed: 13
Completed: 14
Completed: 15
Completed: 16
Completed: 17
Completed: 18
